In [370]:
def plot_confusion_matrix(cm, classes,
                          normalize=False,
                          title='Confusion matrix',
                          cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.
    for i, j in itertools.product(range(cm.shape[0]), range(cm.shape[1])):
        plt.text(j, i, format(cm[i, j], fmt),
                 horizontalalignment="center",
                 color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')

In [390]:
def n_features_range(n_samples=100, m_features=3, m_n_ratio=1):
    '''
    Used for definition of algorithm parameters based on available number of features
     and samples, after feature selection.
     
    It is used for definition of grid search params through a pipeline
     
    :param n_samples: number of samples in the dataframe/matrix
    :param m_features: number of features after feature selection
    :param m_n_ratio: maximal number of features relative to number of samples
    
    :return: range of numbers of features used for param optimization in algorithms 
    '''

    max_features = int(round(n_samples / m_n_ratio))
    if m_features < max_features:
        max_features = m_features

    if m_features == 1:
        min_features = 1
    else:
        min_features = 2

    step_size = int(round(np.log2(max_features)))

    param_range = [a for a in range(min_features, max_features, step_size)]

    if param_range[-1] < max_features:
        param_range.append(max_features)

    return param_range

In [ ]:
'''
Normalization
'''
df=pd.read_csv('~/Downloads/Data-Classification.txt')


scaler=MinMaxScaler() # initialize scaler
X_norm=scaler.fit_transform(X)
pd.DataFrame(X_norm).describe()

# Imports

In [421]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import precision_recall_curve as prc


#from sklearn.model_selection import 
import matplotlib.pyplot as plt

from sklearn.preprocessing import OneHotEncoder



# Read and Prepare Data 

In [422]:
# Read data
df=pd.read_csv('~/Downloads/Data-Classification.txt')

# Create Train and Test Splits
X=df.iloc[:,1:] 
y=df.iloc[:,0] # Separate input and output
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.33, random_state=42) 

In [425]:
y.unique()

array(['A', 'C', 'B'], dtype=object)

# Benchmark Models and Performance

## Algorithms 

In [426]:
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier

In [427]:
benchmark_algorithms ={
    'LR' : LogisticRegression(),
    'RF' : RandomForestClassifier(),
    'KNN': KNeighborsClassifier()
 }

## Performance metrics

In [428]:
from sklearn.metrics.scorer import make_scorer
from sklearn.metrics import roc_auc_score


def mine_roc_auc(y_true, y_pred, average='micro'):
    '''
    Wrapper for AUC type measures (auroc and auprc),
    because they demand input in form of dummy variables
    '''
    y_true_dummy = pd.get_dummies(y_true)
    y_pred_dummy = pd.get_dummies(y_pred)
    return (roc_auc_score(y_true_dummy, y_pred_dummy, average=average))

# Scoring dict to be used in all evaluations
scoring = {
           'au_prc_macro': make_scorer(mine_roc_auc, average='macro'),
           'au_prc_micro': make_scorer(mine_roc_auc, average='macro'),
           'f1_macro': 'f1_macro',
           'f1_micro': 'f1_micro',
           'precision_macro': 'precision_macro',
           'precision_micro': 'precision_micro',
           'accuracy': 'accuracy',
          }

###!!!!!! Micro Macro Explanation https://datascience.stackexchange.com/questions/15989/micro-average-vs-macro-average-performance-in-a-multiclass-classification-settin/16001
### !!! plotting cross val cv

In [429]:
from sklearn.model_selection import cross_validate
#from sklearn.metrics import recall_score

models_dict = {}
scores_dict = {}

for model_name, model in benchmark_algorithms.items():
    
    scores = cross_validate(model, X_train, y_train, scoring=scoring,
                            cv=5, return_train_score=True)
    scores_aggregated= { measure: {'avg': '%.3f' % round(np.average(values),3),\
                                   'std': '%.2f' % round(np.std(values),2)} \
                                    for measure, values in scores.items()}
    scores_dict.update({model_name:scores_aggregated})
    
    fitted_model = model.fit(X_train, y_train)
    models_dict.update({model_name:fitted_model})
    
    



In [430]:
from sklearn.metrics import confusion_matrix
confusions={}
for name, model in models_dict.items():
    y_predicted = model.predict(X_test)
    confusions.update({name: confusion_matrix(y_predicted, y_test)})

In [431]:
confusions['LR']

array([[109,   1,   5],
       [  0, 108,   1],
       [  4,   0, 102]])

In [524]:
scores_dict

{'KNN': {'fit_time': {'avg': '0.003', 'std': '0.00'},
  'score_time': {'avg': '0.041', 'std': '0.00'},
  'test_accuracy': {'avg': '0.394', 'std': '0.04'},
  'test_au_prc_macro': {'avg': '0.546', 'std': '0.03'},
  'test_au_prc_micro': {'avg': '0.546', 'std': '0.03'},
  'test_f1_macro': {'avg': '0.388', 'std': '0.05'},
  'test_f1_micro': {'avg': '0.394', 'std': '0.04'},
  'test_precision_macro': {'avg': '0.397', 'std': '0.04'},
  'test_precision_micro': {'avg': '0.394', 'std': '0.04'},
  'train_accuracy': {'avg': '0.616', 'std': '0.01'},
  'train_au_prc_macro': {'avg': '0.712', 'std': '0.01'},
  'train_au_prc_micro': {'avg': '0.712', 'std': '0.01'},
  'train_f1_macro': {'avg': '0.612', 'std': '0.01'},
  'train_f1_micro': {'avg': '0.616', 'std': '0.01'},
  'train_precision_macro': {'avg': '0.628', 'std': '0.01'},
  'train_precision_micro': {'avg': '0.616', 'std': '0.01'}},
 'LR': {'fit_time': {'avg': '0.102', 'std': '0.01'},
  'score_time': {'avg': '0.013', 'std': '0.00'},
  'test_accurac

# Feature Selection and Parameter Optimization

Since benchmark model (Logistic regression) gave good results in terms of different validation measure (both on training and test data), further analyses will be focused on finding:
- More interpretable solutions
- More robust solutions (in terms of generalization)

For this purpose, hyper-parameters of models will be optimized as well as feature selection

In [432]:
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import RidgeClassifier
from sklearn.linear_model import Lasso

from sklearn.pipeline import Pipeline
from sklearn.decomposition import PCA

In [433]:
# Check if it can be done with simple Pipeline()
pipe = Pipeline([
    ('reduce_dim', PCA()),
    ('classify', RidgeClassifier())
])

In [547]:
def ridge_param_dict(name='classify', estimators=[LogisticRegression()], n_samples=1000, m_features=15): # Check this if needed
    dict={
        name: estimators,
        name + '__' + 'penalty': ['l1', 'l2'],
        name + '__' + 'C': [0.01, 0.03, 0.1, 0.5, 0.8],
        name + '__' + 'max_iter': [10000],
        name + '__' + 'solver': ['saga']
        # check for multinomial
    }
    return (dict)

In [548]:
def rf_param_dict(name='classify', estimators=[RandomForestClassifier()], n_samples=1000, m_features=15):
    dict={
        name: estimators,
        name + '__' + 'n_estimators': range(20, 101, 20),
        name + '__' + 'max_features': n_features_range(n_samples, m_features),
        name + '__' + 'min_samples_leaf': np.arange(0.01, 0.03, 0.05),
        name + '__' + 'max_depth': range(2, 11, 2)
    }
    return (dict)


In [549]:
def create_params_pca_nmf(name='reduce_dim', reducers=[PCA()], n_samples=100, m_features=[5, 10, 15, 20, 25, 30], funcs=[]):
    params=[]

    for func in funcs.values():
        for m in m_features:
            dict = {
                name: reducers,
                name+'__'+'n_components':[m]
            }
            dict.update(func(m_features=m))

            params.append(dict.copy())
    return (params)


In [550]:
algorithms={'Ridge':ridge_param_dict}

In [551]:
pars=create_params_pca_nmf(name='reduce_dim', reducers=[PCA()], n_samples=100, m_features=[5, 10, 15, 20, 25, 30], funcs=algorithms)

In [541]:
#y_train=y_train.map({'A':0, 'B':1, 'C':2})

In [542]:
#y_train=y_train.map({0:'A', 1:'B', 2:'C'})

In [552]:
search=GridSearchCV(estimator = pipe, param_grid = pars, scoring=scoring, refit='au_prc_macro', cv=5, n_jobs=2).fit(X_train, y_train)

In [553]:
components=search.cv_results_
p=search.cv_results_['params'][0] # params for one algorithm

## GridSearchCV log analyses

In [554]:
log_df=pd.DataFrame(components)

In [555]:
log_df.sort_values('mean_test_au_prc_macro', ascending=False).to_csv('results.csv')

In [557]:
log_df.iloc[0]['params']

{'classify': LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
           intercept_scaling=1, max_iter=10000, multi_class='ovr', n_jobs=1,
           penalty='l1', random_state=None, solver='saga', tol=0.0001,
           verbose=0, warm_start=False),
 'classify__C': 0.01,
 'classify__max_iter': 10000,
 'classify__penalty': 'l1',
 'classify__solver': 'saga',
 'reduce_dim': PCA(copy=True, iterated_power='auto', n_components=30, random_state=None,
   svd_solver='auto', tol=0.0, whiten=False),
 'reduce_dim__n_components': 5}

In [566]:
np.set_printoptions(suppress=True)
np.abs(search.best_estimator_.named_steps['classify'].coef_)

array([[ 0.00005194,  0.00005775,  0.00016196,  0.00001041,  0.00029534,
         0.00125713,  0.00056716,  0.00038789,  0.00204169,  0.01705677,
         0.00222576,  0.0235838 ,  0.00325135,  0.00382209,  0.00051098,
         0.01400537,  0.0038145 ,  0.00454965,  0.02261119,  0.00778988,
         0.02423947,  0.05169558,  0.06984935,  0.12177898,  0.1468223 ,
         0.05872214,  0.17039034,  0.04522326,  0.00033544,  0.14801178],
       [ 0.00009962,  0.00006927,  0.00024747,  0.00052031,  0.00004521,
         0.00001639,  0.00120692,  0.00159266,  0.00353399,  0.03443111,
         0.0047229 ,  0.00420293,  0.00556044,  0.01104581,  0.01371967,
         0.00893658,  0.00030236,  0.00496319,  0.07590543,  0.00567939,
         0.08957671,  0.0030803 ,  0.09665548,  0.04637347,  0.13899926,
         0.05623149,  0.00851108,  0.01989131,  0.04998539,  0.06065147],
       [ 0.00023764,  0.00019664,  0.00009421,  0.00034618,  0.00002847,
         0.00061626,  0.00033779,  0.00044141,  0

In [561]:
search.best_estimator_.named_steps['']

Pipeline(memory=None,
     steps=[('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=30, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classify', LogisticRegression(C=0.5, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=10000, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='saga', tol=0.0001,
          verbose=0, warm_start=False))])

In [313]:
pipe.set_params(**components)

Pipeline(memory=None,
     steps=[('reduce_dim', PCA(copy=True, iterated_power='auto', n_components=5, random_state=None,
  svd_solver='auto', tol=0.0, whiten=False)), ('classify', LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False))])

In [315]:
pipe.fit(X_train, y_train).predict(X_test)

array(['C', 'C', 'C', 'C', 'B', 'C', 'C', 'C', 'B', 'C', 'C', 'C', 'B',
       'A', 'B', 'C', 'B', 'C', 'A', 'C', 'B', 'B', 'A', 'B', 'B', 'C',
       'C', 'A', 'C', 'C', 'C', 'A', 'B', 'C', 'B', 'C', 'A', 'C', 'C',
       'B', 'B', 'B', 'B', 'A', 'C', 'A', 'B', 'B', 'B', 'B', 'C', 'C',
       'C', 'B', 'B', 'C', 'C', 'C', 'C', 'B', 'C', 'A', 'A', 'C', 'C',
       'C', 'C', 'A', 'C', 'C', 'A', 'C', 'C', 'C', 'C', 'C', 'C', 'A',
       'C', 'C', 'C', 'C', 'C', 'C', 'B', 'A', 'C', 'C', 'C', 'B', 'C',
       'C', 'C', 'C', 'B', 'B', 'C', 'B', 'C', 'A', 'A', 'B', 'C', 'C',
       'A', 'C', 'C', 'B', 'C', 'C', 'A', 'C', 'C', 'C', 'C', 'C', 'C',
       'C', 'A', 'A', 'C', 'B', 'C', 'B', 'C', 'C', 'C', 'B', 'C', 'C',
       'C', 'C', 'A', 'B', 'A', 'A', 'B', 'B', 'B', 'A', 'B', 'B', 'C',
       'B', 'C', 'A', 'B', 'C', 'C', 'C', 'B', 'C', 'C', 'B', 'A', 'C',
       'C', 'B', 'A', 'B', 'C', 'C', 'C', 'C', 'A', 'C', 'C', 'B', 'C',
       'C', 'B', 'C', 'B', 'A', 'C', 'B', 'C', 'C', 'C', 'C', 'A

In [174]:
search=GridSearchCV(estimator = LogisticRegression(), param_grid = grid, cv=5, n_jobs=2)

In [175]:
search.fit(X_train, y_train)

GridSearchCV(cv=5, error_score='raise',
       estimator=LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False),
       fit_params=None, iid=True, n_jobs=2,
       param_grid={'penalty': ['l1', 'l2'], 'C': [0.01, 0.1, 0.3]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=0)

In [217]:
a=search.cv_results_

In [218]:
a

{'mean_fit_time': array([ 2.4281404 ,  0.08468533,  5.56339583,  0.10868721,  7.47109303,
         0.13296876]),
 'mean_score_time': array([ 0.02096758,  0.00084143,  0.00067897,  0.00078926,  0.00077062,
         0.00096245]),
 'mean_test_score': array([ 0.89552239,  0.94477612,  0.94179104,  0.95522388,  0.94179104,
         0.94925373]),
 'mean_train_score': array([ 0.91642546,  0.9682898 ,  0.95746681,  0.97948315,  0.9597077 ,
         0.98134813]),
 'param_C': masked_array(data = [0.01 0.01 0.1 0.1 0.3 0.3],
              mask = [False False False False False False],
        fill_value = ?),
 'param_penalty': masked_array(data = ['l1' 'l2' 'l1' 'l2' 'l1' 'l2'],
              mask = [False False False False False False],
        fill_value = ?),
 'params': [{'C': 0.01, 'penalty': 'l1'},
  {'C': 0.01, 'penalty': 'l2'},
  {'C': 0.1, 'penalty': 'l1'},
  {'C': 0.1, 'penalty': 'l2'},
  {'C': 0.3, 'penalty': 'l1'},
  {'C': 0.3, 'penalty': 'l2'}],
 'rank_test_score': array([6, 3, 4, 1, 4

In [187]:
LogisticRegression(**a).fit(X_train, y_train)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [188]:
LogisticRegression(**a)

LogisticRegression(C=0.01, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l1', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [145]:
dum_true = pd.get_dummies(y_test)

In [143]:
dum_pred = pd.get_dummies(lr_model.predict(X_test))

In [147]:
roc_auc_score(dum_true, dum_pred, average='micro', sample_weight=None)

0.97500000000000009

In [ ]:
#AUPRC in multilabel settings
#http://scikit-learn.org/stable/auto_examples/model_selection/plot_precision_recall.html

In [ ]:
# Plot precision recall curves for multiclass
"""

from itertools import cycle
# setup plot details
colors = cycle(['navy', 'turquoise', 'darkorange', 'cornflowerblue', 'teal'])

plt.figure(figsize=(7, 8))
f_scores = np.linspace(0.2, 0.8, num=4)
lines = []
labels = []
for f_score in f_scores:
    x = np.linspace(0.01, 1)
    y = f_score * x / (2 * x - f_score)
    l, = plt.plot(x[y >= 0], y[y >= 0], color='gray', alpha=0.2)
    plt.annotate('f1={0:0.1f}'.format(f_score), xy=(0.9, y[45] + 0.02))

lines.append(l)
labels.append('iso-f1 curves')
l, = plt.plot(recall["micro"], precision["micro"], color='gold', lw=2)
lines.append(l)
labels.append('micro-average Precision-recall (area = {0:0.2f})'
              ''.format(average_precision["micro"]))

for i, color in zip(range(n_classes), colors):
    l, = plt.plot(recall[i], precision[i], color=color, lw=2)
    lines.append(l)
    labels.append('Precision-recall for class {0} (area = {1:0.2f})'
                  ''.format(i, average_precision[i]))

fig = plt.gcf()
fig.subplots_adjust(bottom=0.25)
plt.xlim([0.0, 1.0])
plt.ylim([0.0, 1.05])
plt.xlabel('Recall')
plt.ylabel('Precision')
plt.title('Extension of Precision-Recall curve to multi-class')
plt.legend(lines, labels, loc=(0, -.38), prop=dict(size=14))


plt.show()
"""

# Evaluation measures